In [1]:
import pandas as pd
import geopandas as gpd
import rioxarray
import math
import shapely
import xvec

from shapely.geometry import Point, MultiPoint

In [2]:
sightlines = pd.read_pickle("my_results/method_1/d06_sightlines_geometries.pickle")
sightlines = sightlines[["start_point", "end_point", "sight_line_points"]]

# Convert tuples with coordinates to shapely geometry
Should eventually be deprecated by already using shapely geometries
in the first place.

In [3]:
def convert_to_point(row):
    return Point(row)

In [4]:
def convert_to_multipoint(row):
    if row:
        return MultiPoint(row)
    else:
        return None

In [5]:
# Convert start and end point to geometry
tuple_columns = ["start_point", "end_point"]

for col in tuple_columns:
    sightlines[col+"_geom"] = sightlines[col].apply(convert_to_point)


# Convert sight line points to geometry
sightlines["sl_points"] = sightlines["sight_line_points"].apply(convert_to_multipoint)

# Load raster dtm

In [6]:
raster = rioxarray.open_rasterio("data/dtm/dtm_raw/RGEALTI_FXX_1025_6285_MNT_LAMB93_IGN69.asc")
raster = raster.drop_vars("band").squeeze()

# Extract z coordinates from raster

In [7]:
# Extract z coords from raster
z_start = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["start_point_geom"], x_coords="x",y_coords="y")

# Extract z coords from raster
z_end = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["end_point_geom"], x_coords="x",y_coords="y")

In [8]:
# TODO
# This is waayy to complicated. Should be reworked

new_gdf = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry')

for row in sightlines["sl_points"]:
    if row is not None:
        points = row.geoms
        
        z_points = raster.drop_vars("spatial_ref").xvec.extract_points(points=points, x_coords="x",y_coords="y")
        z_points = z_points.xvec.to_geopandas().rename(columns={0: "z"})

        z_points["geometry"] = z_points.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
        z_points = z_points.drop(columns="z")
        
        multipoint = MultiPoint(z_points["geometry"].tolist())
        
        row_gdf = gpd.GeoDataFrame({'geometry': [multipoint]}, geometry='geometry')

        new_gdf = pd.concat([new_gdf, row_gdf], ignore_index=True)
    else:
        new_row = gpd.GeoDataFrame({'geometry': [None]}, geometry='geometry')
        new_gdf = pd.concat([new_gdf, new_row], ignore_index=True)

## Put results together

In [9]:
# Convert do gpd
z_start = z_start.xvec.to_geopandas().rename(columns={0: "z"})

# Append z values to points
z_start["start_point_3d"] = z_start.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
z_start = z_start.drop(columns=["geometry", "z"])

# Convert do gpd
z_end = z_end.xvec.to_geopandas().rename(columns={0: "z"})

# Append z values to points
z_end["end_point_3d"] = z_end.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
z_end = z_end.drop(columns=["geometry", "z"])

In [10]:
sightlines = pd.concat([z_start, z_end, new_gdf], axis=1)

sightlines = sightlines.rename(columns={"start_point_3d": "sl_start",
                                          "end_point_3d": "sl_end",
                                          "geometry": "sl_points"})

In [11]:
sightlines

,sl_start,sl_end,sl_points
0,POINT Z (1025242.3 6281649.6 23.08),POINT Z (1025315.3 6281625.6 24.3),"MULTIPOINT Z ((1025242.675 6281649.27 23.39), ..."
1,POINT Z (1025177.5 6281541.1 20.18),POINT Z (1025169 6281448.7 21.75),"MULTIPOINT Z ((1025177.597 6281540.609 20.18),..."
2,POINT Z (1025424.8 6281581.9 30.91),POINT Z (1025442.8 6281622 33.38),"MULTIPOINT Z ((1025425.005 6281582.356 30.91),..."
3,POINT Z (1025476 6281507.8 33.53),POINT Z (1025538.1 6281459.3 37.83),"MULTIPOINT Z ((1025475.934 6281507.304 33.53),..."
4,POINT Z (1025216 6281670.9 21.13),POINT Z (1025187.8 6281695.8 21.19),"MULTIPOINT Z ((1025215.613 6281671.217 21.13),..."
...,...,...,...
2162,POINT Z (1023809.4 6283978.1 168.65),POINT Z (1023826.5 6283991.1 168.57),MULTIPOINT Z ((1023809.828 6283978.359 168.65)...
2163,POINT Z (1023996.4 6283819.8 179.95),POINT Z (1023991 6283832.7 178.71),MULTIPOINT Z ((1023996.284 6283820.286 179.95)...
2164,POINT Z (1024115.3 6284174.2 187.49),POINT Z (1024132.5 6284154 183.36),MULTIPOINT Z ((1024115.618 6284173.814 187.49)...
2165,POINT Z (1024139.6 6284146.3 178.36),POINT Z (1024213.6 6284091.4 168.23),"MULTIPOINT Z ((1024139.947 6284145.94 178.36),..."


# Compute slope function
Finally, yaaay.

In [15]:
NODATA_RASTER = raster.rio.nodata

def compute_slope(road_row):
    start = road_row.sl_start
    end = road_row.sl_end
    slp = road_row.sl_points

    if slp == None:
        # Case when there is no sight line point (e.g. when the road is too short)
        # just computes slope between start and end
        slope_percent = abs(start.z - end.z) / shapely.distance(start, end)
        slope_degree = math.degrees(math.atan(slope_percent))

        return slope_percent, slope_degree
    
    slp_list = [p for p in slp.geoms]

    points = []

    points.append(start)
    for p in slp_list:
        points.append(p)
    points.append(end)

    # number of points
    nb_points = len([start]) + len([end]) + len(slp_list)

    # temporary variables to store inter slope values
    sum_slope_percent = 0
    sum_slope_radian = 0
    sum_nb_points = 0

    # if theres one or more sight line point
    for i in range (1, nb_points-1):
        a = points[i-1]
        b = points[i+1]
            
        sum_nb_points += 1
        inter_slope_percent = abs(a.z - b.z) / shapely.distance(a, b)

        sum_slope_percent += inter_slope_percent
        sum_slope_radian += math.atan(inter_slope_percent)
        
    # compute mean of inter slopes
    slope_percent = sum_slope_percent/sum_nb_points
    slope_degree = math.degrees(sum_slope_radian/sum_nb_points)

    return slope_degree, slope_percent

# Compute results

In [13]:
slope_values = []

for _, road_row in sightlines.iterrows():

    # display([p for p in road_row.sl_points.geoms])
    # break

    slope_degree, slope_percent = compute_slope(road_row)

    slope_values.append([slope_degree, slope_percent])

    df_slopes = pd.DataFrame(slope_values,
                             columns=["slope_degree",
                                      "slope_percent"])

In [14]:
df_slopes

,slope_degree,slope_percent
0,2.305050,0.040300
1,1.094379,0.019111
2,3.159987,0.055233
3,2.404723,0.042030
4,0.878624,0.015341
...,...,...
2162,0.284191,0.004960
2163,4.664951,0.081627
2164,9.312918,0.166963
2165,6.194412,0.109414
